# Query the Database

### Define dependencies

In [7]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from config import (ServerName, UserName, Password, port, DataBase)
import psycopg2

### Create Engine

In [8]:
rds_connection_string = f'{UserName}:{Password}@{ServerName}:{port}/{DataBase}'
engine = create_engine(f'postgresql://{rds_connection_string}')

### Consult tables in the Database

In [9]:
engine.table_names()

['basic',
 'detail',
 'metrics',
 'geoloc',
 'state_fips',
 'region_id',
 'locale',
 'religious_affiliation',
 'operating',
 'under_investigation',
 'main_campus',
 'institutional_characteristics_level',
 'ownership_peps',
 'ownership',
 'online_only',
 'open_admissions_policy',
 'degrees_awarded_predominant',
 'degrees_awarded_highest',
 'title_iv_eligibility_type',
 'carnegie_size_setting',
 'carnegie_undergrad',
 'carnegie_basic',
 'men_only',
 'women_only']

## Function to Run a SQL File

In [10]:
def run_sql(sql_file, output_csv, output_dic, output_json):

    fd = open(sqlfile, 'r')
    sqlFile = fd.read()
    fd.close()

    # all SQL commands (split on ';')
    sqlCommands = sqlFile.split(';')    

    try:
        # connect to the PostgreSQL server
        engine = create_engine(f'postgres+psycopg2://{UserName}:{Password}@{ServerName}:{port}/{DataBase}')
        conn = engine.connect()
        # create table one by one
        for command in sqlCommands:
            trans = conn.begin()
            conn.execute(command)
            trans.commit()
        # Close connection
        tables = engine.table_names()
        conn.close()
        SQL_Query = pd.read_sql_query(command, con=engine)
        # Create Dataframe
        df = pd.DataFrame(SQL_Query)
        # Create CSV File
        df.to_csv(output_csv, index=False)
        # Create Dictionary
        dict = df.to_dict(orient='records')
        f = open(output_dic,"w")
        f.write( str(dict) )
        f.close()
        # Create JSON file
        json = df.to_json(orient='records')
        f = open(output_json,"w")
        f.write( str(json) )
        f.close()
        print('Dataframe, csv, txt, and json files were created')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

## Run the SQL file with the Full Query

In [11]:
# Input routes to the output files and the source of the SQL code
sqlfile = 'queries/full_query.sql'
outputcsv = 'output/full_query.csv'
outputdic = 'output/full_query_dict.txt'
outputjson = 'output/full_query_json.json'
# Run the function to query the db and create output files
run_sql(sqlfile, outputcsv, outputdic, outputjson)
# Read and print csv file just created from the database
data = pd.read_csv(outputcsv)
data.head()

Dataframe, csv, txt, and json files were created


,id,ope8_id,ope6_id,name,alias,city,state,state_fips_dec,zip,region_id_desc,...,degree_urbanization,carnegie_size_setting_desc,carnegie_undergrad_desc,carnegie_basic_desc,men_only_desc,women_only_desc,tuition_in_state,tuition_out_of_state,location_lon,location_lat
0,215974,657700,6577,University of Pittsburgh Medical Center-Shadys...,NaN,Pittsburgh,PA,Pennsylvania,15232,"Mid East (DE, DC, MD, NJ, NY, PA)",...,NaN,Not applicable,Not applicable,Not applicable,No,No,15315.0,15315.0,-79.9395,40.4548
1,212391,1054203,10542,Empire Beauty School-Reading,NaN,Reading,PA,Pennsylvania,19605,"Mid East (DE, DC, MD, NJ, NY, PA)",...,NaN,Not applicable,Not applicable,Not applicable,No,No,NaN,NaN,-75.9275,40.3702
2,212294,966411,9664,Empire Beauty School-Shamokin Dam,NaN,Shamokin Dam,PA,Pennsylvania,17876,"Mid East (DE, DC, MD, NJ, NY, PA)",...,NaN,Not applicable,Not applicable,Not applicable,No,No,NaN,NaN,-76.8159,40.8546
3,212267,1054205,10542,Empire Beauty School-York,NaN,York,PA,Pennsylvania,17402,"Mid East (DE, DC, MD, NJ, NY, PA)",...,NaN,Not applicable,Not applicable,Not applicable,No,No,NaN,NaN,-76.6747,39.9725
4,221634,2092800,20928,Tennessee College of Applied Technology-Whitev...,TCAT Whiteville,Whiteville,TN,Tennessee,38075-0489,"Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC,...",...,NaN,Not applicable,Not applicable,Not applicable,No,No,NaN,NaN,-89.1594,35.3196
